# Validierung des Rainbow-Monte-Carlo Pricers 0.1
## Bewertung von Plain Vanilla Options im Black-Scholes-Modell


In [ ]:
import pyvacon.analytics as analytics         #Pricing class
import pyvacon.tools.enums as pte             #Enums class, contains inter-/extrapolation types, daycount conventions, etc. 
import pyvacon.marketdata.plot as mkt_plt     #Plot class for marketdata, e.g. discount curves, vol surfaces, etc.

import datetime as dt                         #Python class for date and time handling
import matplotlib.pyplot as plt               #Python plot class
import numpy as np                            #Python class for numerical, vector/matrix based programming
import math                                   #Python class for basic mathematical operations
%matplotlib inline                            

## Plain Vanilla Options
Ein Plain-Vanilla Call auf ein Underlying $S$ mit Maturity $T$ und Strike $K$ hat das folgende Auszahlungsprofil
$(S(T)-K)_+$ und dementsprechend einen fairen Preis $\text{Price}_{C}(S,t) = \mathbb{E}[(S(T)-K)_+]$. Im Black-Scholes-Modell 
lässt sich dieser Preis analytisch bestimmen
### Black-Scholes-Formel
$$\text{Price}_{C/P}(S,t) = \pm \bigg(S \Phi\big(\pm d_1\big)-K e^{-r(T-t)}\Phi\big(\pm d_2\big)\bigg),\qquad d_{1/2} = \frac{\log(S/K)+(r\pm\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}}.$$

In [ ]:
def ndf(x): 
#Normalverteilungsfunktion
    return 0.5*(1+math.erf(x/math.sqrt(2)))

In [ ]:
def Blsch(S,K,t,T,sigma,r,cp): 
#Black-Scholes-Formel
    d1 = (math.log(S/K)+(r+sigma*sigma/2)*(T-t))/(sigma*math.sqrt(T-t))
    d2 = d1-sigma*math.sqrt(T-t)
    return cp*(S*ndf(cp*d1)-K*math.exp(-r*(T-t))*ndf(cp*d2))

### Inputparameter der Black-Scholes Formel
- volatility:     $\sigma$
- interest rate:  r
- strike:         K
- expiry:         T

### Variablen in Black-Scholes Formel
- Spot zum Zeitpunkt $t$: S
- Auswertungszeitpunkt: $t$

In [ ]:
S = 50
sigma = 0.3
r = 0.05
K = 50
t = 0
T = 0.5

In [ ]:
    refdate_p = dt.datetime(2018,1,1)
    expiry_p = refdate_p + dt.timedelta(days = 365*(T-t))
    refdate = analytics.ptime(refdate_p.year,refdate_p.month,refdate_p.day,0,0,0)
    expiry = analytics.ptime(expiry_p.year,expiry_p.month,expiry_p.day, 0, 0, 0)
# Discountkurve deklarieren
    dc = analytics.DiscountCurve('',refdate,[refdate,analytics.ptime(2021,1,1,0,0,0)], [1.0, 1/math.pow(1+r,3)], 
                                 pte.DayCounter.ACT365_FIXED, pte.InterpolationType.LINEAR, pte.ExtrapolationType.NONE)
# Borrowing Curve deklarieren
    bc = analytics.DiscountCurve('',refdate,[refdate,analytics.ptime(2021,1,1,0,0,0)], [1.0, 1], 
                                 pte.DayCounter.ACT365_FIXED, pte.InterpolationType.LINEAR, pte.ExtrapolationType.NONE)
# Dividenden Zahlungen definieren
    div=analytics.DividendTable('', refdate, analytics.vectorPTime(), [],[],[],[])
# Forwardkurve definieren
    fwd = analytics.EquityForwardCurve(refdate, S, dc, bc, div)
# Volatility Surface definieren
    flat_param = analytics.VolatilityParametrizationFlat(sigma)
    ?analytics.VolatilityParametrizationSSVI
    vol_srf = analytics.VolatilitySurface('',refdate, fwd, pte.DayCounter.ACT365_FIXED, flat_param)

In [ ]:
# Definition des Baskets und des Zahlungsprofils des Derivates

?analytics.RainbowUnderlyingSpec
rbw_udl = analytics.vectorRainbowUdlSpec(1)
rbw_udl[0] = analytics.RainbowUnderlyingSpec(['dummy'], [1.0], [100000000.0], [0.0], [1.0], [1.0], 10000000.0, 0.0, 0, 'None', 'None',
                                            [], 0, 'None', [], 'None')
#?analytics.RainbowBarrierSpec

barriers = analytics.vectorRainbowBarrierSpec(1)
barriers[0] = analytics.RainbowBarrierSpec(expiry, expiry, [], 0, 100000000, 0, [],[], True, [0.0, K, 100000000.0], 
                                           [00.0, 0.0, 100000000.0-K], [],[], expiry)
#?analytics.RainbowSpecification
spec = analytics.RainbowSpecification('', '', pte.SecuritizationLevel.NONE, 'EUR', expiry, barriers, rbw_udl)

In [ ]:
pricing_data = analytics.LocalVolMonteCarloPricingData()

corr = analytics.vectorVectorDouble(1)
corr[0] = analytics.vectorDouble(1, 1.0)
pricing_data.setCorrelations(corr)
pricing_data.param = analytics.MonteCarloPricingParameter()
pricing_data.pricingRequest = analytics.PricingRequest()
pricing_data.valDate = refdate
pricing_data.dsc = dc
pricing_data.spec = spec
pricing_data.vols = analytics.vectorConstVolatilities([vol_srf])
#analytics.price()

In [ ]:
ref_value = Blsch(S,K,t,T,sigma,r,-1)
print(ref_value)

### Konvergenz bezüglich MC Sample

In [ ]:
n = 21
results = np.zeros((n,1))
err = np.zeros((n,1))
mc_samples = np.zeros((n,1))
for i in range(0,n):
    mc_samples[i,0]=2**i
    pricing_data.param.mcParam.numberOfSimulations = 2**i
    pricingResults = analytics.price(pricing_data)
    print(pricingResults.getPrice())
    results[i,0] = pricingResults.getPrice()
    err[i,0] = math.fabs(ref_value - pricingResults.getPrice())


In [ ]:
plt.loglog(mc_samples,err)
plt.show()
pricing_data.param.mcParam.numberOfSimulations = 10000

### Sensitivität bezüglich Spot

In [ ]:
Spots = np.array(range(90,111,2))
n = Spots.size
Prices = np.zeros((n,1))
BS_Prices = np.empty((n,1))
for i in range(0,n):
    S = float(Spots[i])
    fwd = analytics.EquityForwardCurve(refdate, S, dc, bc, div)
    flat_param = analytics.VolatilityParametrizationFlat(sigma)
    vol_srf = analytics.VolatilitySurface('',refdate, fwd, pte.DayCounter.ACT365_FIXED, flat_param)
    pricing_data.vols = analytics.vectorConstVolatilities([vol_srf])
    pricingResults = analytics.price(pricing_data)
    print(pricingResults.getPrice())
    Prices[i,0] = pricingResults.getPrice()
    BS_Prices[i,0] = Blsch(S,K,t,T,sigma,r,1)
    print(BS_Prices[i,0])

In [ ]:
plt.plot(Spots, BS_Prices)

In [ ]:
    S = 100
    fwd = analytics.EquityForwardCurve(refdate, S, dc, bc, div)
    flat_param = analytics.VolatilityParametrizationFlat(sigma)
    vol_srf = analytics.VolatilitySurface('',refdate, fwd, pte.DayCounter.ACT365_FIXED, flat_param)
    pricing_data.vols = analytics.vectorConstVolatilities([vol_srf])

### Sensitivität bzgl Strike

In [ ]:
Strikes = np.array(range(90,111,2))
n = Strikes.size
Prices_2 = np.zeros((n,1))
for i in range(0,n):
    K = float(Strikes[i])
    barriers[0] = analytics.RainbowBarrierSpec(expiry, expiry, [], 0, 100000000, 0, [],[], True, [0.0, K, 100000000.0], 
                                               [00.0, 0.0, 100000000.0-K],[],[], expiry)
    spec = analytics.RainbowSpecification('', '', pte.SecuritizationLevel.NONE, 'EUR', expiry, barriers, rbw_udl)
    pricing_data.spec = spec
    pricingResults = analytics.price(pricing_data)
    print(pricingResults.getPrice())
    Prices_2[i,0] = pricingResults.getPrice()

In [ ]:
plt.plot(Strikes, Prices_2)

In [ ]:
    K = 100
    barriers[0] = analytics.RainbowBarrierSpec(expiry, expiry, [], 0, 100000000, 0, [],[], True, [0.0, K, 100000000.0], 
                                               [00.0, 0.0, 100000000.0-K],[],[], expiry)
    spec = analytics.RainbowSpecification('', '', pte.SecuritizationLevel.NONE, 'EUR', expiry, barriers, rbw_udl)
    pricing_data.spec = spec

### Sensitivität bezüglich Volatilität

In [ ]:
print(S)
K=100
print(sigma)
print(T)

In [ ]:
Sigmas = np.linspace(0.01,20,20)
n = Sigmas.size
Prices_3 = np.zeros(n)
Prices_Ref = np.zeros(n)
for i in range(0,n):
    sigma = Sigmas[i]
    flat_param = analytics.VolatilityParametrizationFlat(sigma)
    vol_srf = analytics.VolatilitySurface('',refdate, fwd, pte.DayCounter.ACT365_FIXED, flat_param)
    pricing_data.vols = analytics.vectorConstVolatilities([vol_srf])
    pricingResults = analytics.price(pricing_data)
    print(pricingResults.getPrice())
    Prices_3[i] = pricingResults.getPrice()
    Prices_Ref[i] = Blsch(S,K,t,T,sigma,r,1)

In [ ]:
plt.plot(Sigmas,Prices_3)